In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [3]:
import pprint
import pickle
import glob
import random
from pathlib import Path
import os
import typing

import torch
from tqdm.autonotebook import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# need pip install open3d
import open3d
import plotly.express as px
import plotly

from utils import general_utils
from utils import eval_utils

In [4]:
def plot_pointcloud(pointcloud, normalized_range=None):
    plot_color = False
    if pointcloud.shape[-1] == 4:
        plot_color = True
    if normalized_range is not None:
        if plot_color:
            fig = px.scatter_3d(x=pointcloud[:,0], y=pointcloud[:,1], z = pointcloud[:,2], color=pointcloud[:,3], opacity=0.7, range_x=normalized_range, range_y=normalized_range, range_z=normalized_range)
        else:
            fig = px.scatter_3d(x=pointcloud[:,0], y=pointcloud[:,1], z = pointcloud[:,2], opacity=0.7, range_x=normalized_range, range_y=normalized_range, range_z=normalized_range)
    else:
        if plot_color:
            fig = px.scatter_3d(x=pointcloud[:,0], y=pointcloud[:,1], z = pointcloud[:,2], color=pointcloud[:,3], opacity=0.7)
        else:
            fig = px.scatter_3d(x=pointcloud[:,0], y=pointcloud[:,1], z = pointcloud[:,2], opacity=0.7)

    # based on https://stackoverflow.com/questions/61371324/scatter-3d-in-plotly-express-colab-notebook-is-not-plotting-with-equal-axes
    fig.update_layout(scene=dict(aspectmode="cube"))
    fig.update_traces(marker=dict(size=5, line={"width":1}))
    fig.show()

In [5]:
# eval fscore, based on code from https://github.com/lmb-freiburg/what3d
def calculate_fscore(gt: open3d.geometry.PointCloud, pr: open3d.geometry.PointCloud, th: float=0.01) -> typing.Tuple[float, float, float]:
    '''Calculates the F-score between two point clouds with the corresponding threshold value.'''
    #d1 = open3d.geometry.compute_point_cloud_to_point_cloud_distance(gt, pr)
    #d2 = open3d.geometry.compute_point_cloud_to_point_cloud_distance(pr, gt)
    
    d1 = gt.compute_point_cloud_distance(pr)
    d2 = pr.compute_point_cloud_distance(gt)
    
    if len(d1) and len(d2):
        recall = float(sum(d < th for d in d2)) / float(len(d2))
        precision = float(sum(d < th for d in d1)) / float(len(d1))

        if recall+precision > 0:
            fscore = 2 * recall * precision / (recall + precision)
        else:
            fscore = 0
    else:
        fscore = 0
        precision = 0
        recall = 0

    return fscore, precision, recall

In [45]:
class_name = "04530566"
refined_meshes_dir = "data/refinements/shapenet_occnet_refinements/gt_pose/{}".format(class_name)
recompute = False

n_points_sample = 5000

eval_fscore_output_path = os.path.join(refined_meshes_dir, "f_score_eval_results.pkl")

In [46]:
# preparing gt shapes paths
#top_level_cfg_path = general_utils.get_top_level_cfg_path(refined_meshes_dir, "default.yaml")
top_level_cfg_path = os.path.join(refined_meshes_dir, "{}.yaml".format(class_name))
cfg = general_utils.load_config(top_level_cfg_path)
gt_shapes_list_path = cfg["dataset"]["gt_shapes_lst_path"]
gt_shapes_dict = {}
with open(gt_shapes_list_path, 'r') as f:
    f = f.read().split('\n')
    for line in f:
        if line != "":
            gt_shapes_dict[line.split(" ")[0]] = line.split(" ")[1]

# preparing original reconstruction paths
original_meshes_dir = cfg["dataset"]["input_dir_mesh"]
original_reconstructions_dict = {str(filename).split('/')[-1].split('.')[0]:str(filename) for filename in list(Path(original_meshes_dir).rglob('*.obj'))}

# preparing refined reconstructions paths
refined_reconstructions_dict = {str(filename).split('/')[-1].split('.')[0]:str(filename) for filename in list(Path(refined_meshes_dir).rglob('*.obj'))}

CUBE_SIDE_LEN = 1.0
threshold_list = [CUBE_SIDE_LEN/200, CUBE_SIDE_LEN/100,
                  CUBE_SIDE_LEN/50, CUBE_SIDE_LEN/20]

In [47]:
def get_pc_from_mesh_path(mesh_path, n_points_sample):
    mesh = open3d.io.read_triangle_mesh(mesh_path)
    # check if mesh has vertices:
    if np.asarray(mesh.vertices).shape[0] == 0:
        return None
    else:
        mesh_pc = mesh.sample_points_uniformly(number_of_points=n_points_sample)
        return mesh_pc

In [50]:
# TODO: also try sampling points poisson disk?

if os.path.exists(eval_fscore_output_path) and not recompute:
    f_score_results = pickle.load(open(eval_fscore_output_path, "rb"))
else:
    f_score_results = pd.DataFrame()

zero_verts_errors = []
for instance in tqdm(refined_reconstructions_dict):
    if f_score_results.empty or instance not in list(f_score_results["instance"]):
        # loading pointclouds
        gt_mesh_path = gt_shapes_dict[instance]
        gt_pc = get_pc_from_mesh_path(gt_mesh_path, n_points_sample)
        #plot_pointcloud(np.asarray(gt_mesh_pc.points))
        
        original_mesh_path = original_reconstructions_dict[instance]
        original_pc = get_pc_from_mesh_path(original_mesh_path, n_points_sample)
        
        refined_mesh_path = refined_reconstructions_dict[instance]
        refined_pc = get_pc_from_mesh_path(refined_mesh_path, n_points_sample)
        if refined_pc is None: # checking for no vertices
            print("No verts found for {}".format(refined_mesh_path))
            zero_verts_errors.append(instance)
            
            instance_result_dict = {"instance":instance}
            for th in threshold_list:
                instance_result_dict["th_{}_pc".format(th)] = np.inf
            f_score_results = f_score_results.append(instance_result_dict, ignore_index=True)
            continue
        
        
        instance_result_dict = {"instance":instance}
        for th in threshold_list:
            original_rec_f_score, precision, recall = calculate_fscore(gt_pc, original_pc, th=th)
            refined_rec_f_score, precision, recall = calculate_fscore(gt_pc, refined_pc, th=th)
            if original_rec_f_score != 0:
                instance_result_dict["th_{}_pc".format(th)] = int(100*(refined_rec_f_score-original_rec_f_score)/original_rec_f_score) # TODO: change to round
            else:
                instance_result_dict["th_{}_pc".format(th)] = np.inf
        f_score_results = f_score_results.append(instance_result_dict, ignore_index=True)
        f_score_results.to_pickle(eval_fscore_output_path)

print("zero_verts_errors: {}".format(zero_verts_errors))

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D IN

[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D IN

[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D IN

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2

zero_verts_errors: []


In [63]:
class_names = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
shapenet_id_to_name = {"02691156":"Airplane", "02828884":"Bench", "02933112":"Cabinet", "02958343":"Car", "03001627":"Chair", "03211117":"Display", "03636649":"Lamp",
                       "03691459":"Speakers", "04090263":"Rifle", "04256520":"Sofa", "04379243":"Table", "04401088":"Telephone", "04530566":"Watercraft"}

total_avgs = {"th_{}_pc".format(th):0.0 for th in threshold_list}

for class_name in class_names:
    refined_meshes_dir = "data/refinements/shapenet_occnet_refinements/gt_pose/{}".format(class_name)
    eval_fscore_output_path = os.path.join(refined_meshes_dir, "f_score_eval_results.pkl")
    if os.path.exists(eval_fscore_output_path):
        f_score_results = pickle.load(open(eval_fscore_output_path, "rb"))
        f_score_results_mean = f_score_results.replace([np.inf, -np.inf], np.nan).mean(skipna=True)
        f_score_results_mean_dict = f_score_results_mean.to_dict()
        print("{}: {}".format(shapenet_id_to_name[class_name], f_score_results_mean_dict))
        for key in f_score_results_mean_dict:
            total_avgs[key]+=f_score_results_mean_dict[key]
            
total_avgs = {key:total_avgs[key]/len(class_names) for key in total_avgs}
print("\nmean    : {}".format(total_avgs))

Airplane: {'th_0.005_pc': 32.180225281602006, 'th_0.01_pc': 28.68375, 'th_0.02_pc': 20.950124688279303, 'th_0.05_pc': 6.232298136645963}
Bench: {'th_0.005_pc': 64.19117647058823, 'th_0.01_pc': 89.71052631578948, 'th_0.02_pc': 67.61516034985422, 'th_0.05_pc': 27.060693641618496}
Cabinet: {'th_0.005_pc': 14.942528735632184, 'th_0.01_pc': 49.9010989010989, 'th_0.02_pc': 77.83243243243243, 'th_0.05_pc': 4.7368421052631575}
Car: {'th_0.005_pc': 39.072874493927124, 'th_0.01_pc': 60.79291917167669, 'th_0.02_pc': 21.099399599733154, 'th_0.05_pc': 4.701801200800534}
Chair: {'th_0.005_pc': 41.78234854151084, 'th_0.01_pc': 16.918535127055307, 'th_0.02_pc': 6.46975354742345, 'th_0.05_pc': 12.426656738644825}
Display: {'th_0.005_pc': 43.455399061032864, 'th_0.01_pc': 33.8287037037037, 'th_0.02_pc': 16.51388888888889, 'th_0.05_pc': 8.577981651376147}
Lamp: {'th_0.005_pc': 54.802850356294535, 'th_0.01_pc': 156.40941176470588, 'th_0.02_pc': 58.127610208816705, 'th_0.05_pc': 45.470319634703195}
Speaker

In [61]:
(28+89+49+30+16+33+156+13+129+15+54+32+40+55)/13



56.84615384615385